In [5]:
from watsonx import WatsonxAI
import warnings
from colorama import Fore, Back

warnings.filterwarnings("ignore")

from dotenv import load_dotenv
from os import getenv

load_dotenv()
BRAVE_API_KEY = getenv("BRAVE_API_KEY", None)


proxy = "proxy.us.ibm.com:8080"

wx = WatsonxAI()
wx.connect()

In [4]:
def gen_naive_answer(question):
    """
    get the AI answer the question directly.
    """

    prompt = f"""<|user|>
please answer the question.
<<SYS>>
question: `{question}`
<</SYS>>
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

answer = gen_naive_answer("how many r in strawberry?")
print(answer)

The word "strawberry" contains one 'r'.


# 1) reflection

In [ ]:
def gen_review(draft):
    prompt = f"""<|user|>
please review the draft provided.
-check if any illogical content.
-check if any unclear content.
-give contructive suggestion.
draft: `{draft}`
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

def gen_revise(content,review):
    prompt = f"""<|user|>
please revise the content base on the review provided.
-well adopt the suggestion provided.
-generate the revised content only.
content: `{content}`
review: `{review}`
<|assistant|>
revised version:
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

requirement = "plan a trip to jakarta for 3 days"

draft = gen_naive_answer(requirement)
print("draft:")
print(Fore.BLUE, draft)

review = gen_review(draft)
print("review:")
print(Fore.RED, review)

revise = gen_revise(draft,review)
print("revise:")
print(Fore.YELLOW, revise)

draft:
 Sure, here's a basic plan for your trip to Jakarta:

Day 1:
- Start your day with a visit to the National Monument (Monas), which is a significant landmark in Jakarta.
- Next, head to the Istiqlal Mosque, one of the largest mosques in Southeast Asia.
- In the afternoon, explore the Fatahillah Square, a historic area with colonial architecture and local markets.
- End your day with a visit to the Ancol Beach, where you can enjoy the sunset and local seafood.

Day 2:
- Begin your day at the Jakarta Old Town (Kota Tua), where you can see Dutch colonial architecture and visit the Museum of Fine Arts and Ceramics.
- After lunch, head to the Jakarta Cathedral and the nearby Chinatown for shopping and food.
- In the evening, enjoy a traditional Wayang Kulit puppet show.

Day 3:
- Spend your morning at the Jakarta Zoo or the Ragunan Zoo, depending on your preference.
- After lunch, visit the Taman Mini Indonesia Indah, a cultural park that showcases various regions of Indonesia.
- End 

# Pattern 1: Planning

In [ ]:
def gen_thought(requirement):
    """
    get the AI not jump to conclusion or action, try approach a problem with thought first.
    """

    prompt = f"""<|system|>
you are a chatbot that answer general question.
-understand the requirement in detail.
-list out what knowlege you need to answer the requirement.
-take the simplest solution.
-classify what domain this requirement about.
-think about where will the right source of the answer.
-think logically.
-think step by step.
-you trust the result of the tools more than your memory.
-you trust the knowledge provide more than your memory.
-leverage tool if possible.
-dont guess.
-dont overcomplicate.
-declare the assumption you made.
-NEVER generate Final Result
<|user|>
please generate your thought on the question provided.
<<SYS>>
requirement: `{requirement}`
<</SYS>>
<|assistant|>
thought:
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

requirement = "how many r in strawberry?"
thought = gen_thought(requirement)
print(thought)

In [ ]:
def gen_plan(requirement):
    """
    get the API to break down the big task to smaller sub-tasks. that easier to execute
    """
    
    prompt = f"""<|system|>
-dont overcomplicate.
-think logically.
-think step by step.
-dont guess.
-you trust the result of the tools more than your memory.
-you trust the knowledge provide more than your memory.
-you trust the status provided. no need to check on it.
<|user|>
please generate a concise and solid plan to satisfy the requirement provided.
-understand the requirement in detail.
-understand the requirement with common sense.
-understand the status provided.
-list out what knowlege you need to satisfy the requirement.
-leverage tool if possible.
-please breakdown complex task to simple tasks.
<<SYS>>
requirement: `{requirement}`
<</SYS>>
-DONT generate result of the steps.
-DONT generate status.
-**DONT** generate **Final Result:**
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

def gen_next_step(plan,status):
    """
    get the API to break down the big task to smaller sub-tasks. that easier to execute
    """
    
    prompt = f"""<|system|>
<|user|>
please check the status against the plan provided.
-understand the plan provided in detail.
-understand the status provided in detail.
-generate next step base on status if has next step.
-print 'complete' if no next step.
<<SYS>>
plan: `{plan}`
status: `{status}`
output format: {{
    next_step:,
    complete: yes/no,
}}
<</SYS>>
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

plan = gen_plan("goto Jakarta from Singapore")
print("plan:")
print(plan)

next_step = gen_next_step(plan,"I have the visa and passport")
print("next step:")
print(next_step)

next_step = gen_next_step(plan,"already in Jakarta")
print("next step:")
print(next_step)

plan:
1. Research the best mode of transportation to travel from Singapore to Jakarta, considering factors such as cost, time, and convenience.
2. Check the availability and schedules of flights, trains, or buses between the two cities.
3. Book a ticket for the chosen mode of transportation, ensuring it aligns with your schedule and budget.
4. Prepare necessary documents, such as passports or identification cards, for the journey.
5. Pack appropriate clothing and essentials for the trip, considering the weather and duration of the journey.
6. Arrange for any necessary accommodations in Jakarta, such as hotels or hostels.
7. Plan for any additional expenses, such as meals, local transportation, or activities during the trip.
8. Confirm all bookings and arrangements before departure.
9. Travel to the designated pick-up point or airport/train station/bus terminal for your journey.
10. Arrive at your destination in Jakarta and proceed with your planned activities.
next step:
{
    "next_st

# Pattern 3: Use Tool

In [8]:
import inspect
import ast
import json

def get_function_signature_and_docstring(func):
    source_lines = inspect.getsourcelines(func)[0]
    source = ''.join(source_lines)

    # Parse the function definition using AST (Abstract Syntax Trees)
    parsed_func = ast.parse(source)

    # Extract function signature and docstring
    func_signature = ''
    docstring = ''
    return_type = ''
    for node in parsed_func.body:
        if isinstance(node, ast.FunctionDef):
            func_signature = source_lines[node.lineno - 1].strip()
            docstring = ast.get_docstring(node) or ''
            # Get return type annotation
            return_type = node.returns.id if node.returns else 'None'
            break

    return func_signature, docstring, return_type

def format_function_info(func):
    func_signature, docstring, return_type = get_function_signature_and_docstring(func)
    function_name = func.__name__

    # Extract parameters from a JSON schema-like structure
    parameters = {
        "type": "object",
        "properties": {}
    }

    # Get the function's parameters and their annotations
    func_params = inspect.signature(func).parameters
    for param_name, param in func_params.items():
        param_type = param.annotation
        param_description = ""  # You can add a description here if needed
        parameters["properties"][param_name] = {
            "type": str(param_type),
            "description": param_description
        }

    # Set the required parameters based on function annotations
    required_params = [param_name for param_name, param in func_params.items() if param.default == inspect.Parameter.empty]
    parameters["required"] = required_params

    return {
        "name": function_name,
        "parameters": parameters,
        "description": docstring,
        "return_type": return_type
    }

def get_tools_prompt(tools):
    prompt = "["
    # Get the source code of the function
    for tool in tools:
        info = format_function_info(tool)
        prompt = prompt + f"{info}\n"
    prompt = prompt + "]"
    return prompt

def pick_tool(job,thought,tools):
    """
    pick a tool from toolbox, that best match the need for requirement.
    """

    prompt = f"""<|system|>
You are a helpful assistant with access to the following function calls. 
- Your task is to produce a sequence of function calls necessary to generate response to the user utterance. 
- well understand the thought to articulate the task.
- well understand the description of tools.
- only pick the function required and provided in tools.
- show "missing tool" if no appropriate function provided.
- DONT explain.
output format: [{{"name": "function", "arguments": {{"arg": "value"}}}}]
tools: {get_tools_prompt(tools)}
<|user|>
thought: `{thought}`
job: `{job}`
<|assistant|>
"""
    # print(prompt)
    # print(">>"+step)
    tool = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return tool

def use_tool(func_json_str):
    """
    you already hv the tool, let run it.
    """
    try:
        # Parse the JSON string into Python objects
        function_calls = json.loads(func_json_str)
        
        # Dictionary to store results
        results = {}
        
        # Iterate over each function call in the list
        for call in function_calls:
            func_name = call.get("name")
            args = call.get("arguments", {})
            print(f"calling {func_name}({args})...")
            
            # Check if the function exists in the global namespace
            if func_name in globals() and callable(globals()[func_name]):
                try:
                    # Retrieve the function object
                    func = globals()[func_name]
                    
                    # Call the function with the provided arguments
                    result = func(**args)
                    
                    # Store the result
                    results[f"{func_name}({args})"] = result
                except Exception as e:
                    # Handle exceptions raised by the function
                    results[f"{func_name}({args})"] = f"Error executing {func_name}: {str(e)}"
            else:
                # Function not found or not callable
                results[f"{func_name}({args})"] = f"Function '{func_name}' not found or is not callable."
        
        return results
    
    except json.JSONDecodeError as jde:
        return {"JSONDecodeError": f"Invalid JSON input: {str(jde)}"}
    except Exception as ex:
        return {"Error": f"An unexpected error occurred: {str(ex)}"}

In [9]:
import json5

def call_func(function_info):
    # Use globals to call the function
    function_name = function_info['name']
    parameters = function_info['parameters']
    print(f"calling {function_name}({parameters})...")

    # Call the function using globals
    if function_name in globals():
        return globals()[function_name](**parameters)
    else:
        return (f"Function {function_name} not found.")

def gen_call_tool(task,thought,tools,debug=False):
    prompt = f"""<|system|>
please generate a sequence of tool calling get the knowledge that help your task.
-well understand the thought to articulate the task.
-well understand the description of tools.
-breakdown the task provided.
-generate appropriate parameter base on the tool definition.
-please use the tools provided only.
-only pick the tool helpful for your answer.
-dont pick unrelated tool.
-dont makeup tools if not available.
<<TOOLS>>
tools: `{get_tools_prompt(tools)}`
<</TOOLS>>
task: `{task}`
thought: `{thought}`
-please follow the output format.
output format: `[
    {{
        "name": "", 
        "parameters": {{
            "param name": "param value",
            }}
     }}
]`
<|assistant|>
···jsonl:"""

    if debug:
        print(f"prompt = {prompt}")

    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT).replace("```","").replace("<|assistant|>","").replace("<|endofjson|>","")
    if debug:
        print(f"|{answer}|")

    output = []
    function_calls = json5.loads(answer)
    for function_call in function_calls:
        output.append(call_func(function_call))
    # print(output)
    return output

In [10]:
def gen_summary_from_knowledge(question,knowledge):
    prompt = f"""<|user|>
help answer the question provided base on the knowledge provided.
-focus on what you found instead of not found.
-if no answer provide, just say you dont know.
<<SYS>>
question = `{question}`
knowledge = `{knowledge}`
<</SYS>>
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT,8000)
    return answer
    
def gen_answer_from_knowledge(question,tools,debug=False):
    thought = gen_thought(question)
    knowledge = gen_call_tool(question,thought,tools,debug)
    if debug:
        print(f"knowledge:{knowledge}")

    answer = gen_summary_from_knowledge(question,knowledge)
    return answer

In [2]:
import requests
import wikipediaapi
import requests
import yfinance as yf

def search_engine(query)->str:
    """
    it is a search engine that take query as input and return results a long string.
    it take one parameter as query.
    """

    # Define the API endpoint URL
    url = "https://api.search.brave.com/res/v1/web/search"

    # Define the query parameters
    params = {
        "q": query
    }

    # Define the headers
    headers = {
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "X-Subscription-Token": BRAVE_API_KEY
    }

    # Make the GET request
    response = requests.get(url, params=params, headers=headers)

    news_descriptions = []

    # return "hello"

    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        news_descriptions = data
        
        news_descriptions = [result.get('description', '') for result in data.get('web', {}).get('results', [])]

    else:
        # Print an error message if the request was not successful
        print("Error: Failed to retrieve data from the API. Status code:", response.status_code)
        news_descriptions = response.status_code
    
    return str(news_descriptions)

def query_wikipedia(topic)->str:
    """
    get detail definition and information of particular topic
    only very short keyword/topic will be support. please use appropriate one.
    """
    wiki = wikipediaapi.Wikipedia('watsonx',proxies={'http': proxy,'https':proxy})
    page = wiki.page(topic)

    if page.exists():
        return {
            "title": page.title,
            "summary": page.summary,
            "url": page.fullurl
        }
    else:
        return "Page not found."

def get_stock_data(ticker, period='5d', interval='1d')->str:
    """
    get stock price. it is for stock data only.
    """
    session = requests.Session()
    session.proxies = {
        'http': proxy,
        'https': proxy
    }

    stock = yf.Ticker(ticker, session=session)
    data = stock.history(period=period, interval=interval)
    return data

def query_search_engine(query):
    """
    search any information thru search engine.
    input any query should be fine.
    """
    url = 'https://api.duckduckgo.com/'
    params = {
        'q': query,
        'format': 'json',
        'pretty': 1
    }
    headers = {
        'User-Agent': 'YourAppName/1.0 (https://yourapp.example; your-email@example.com)'
    }

    session = requests.Session()
    session.headers.update(headers)

    if proxy:
        session.proxies = {
            'http': proxy,
            'https': proxy
        }

    response = session.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        print(data)
        return data.get('Result', [])
    else:
        return f"Error: {response.status_code}"
    
import pytz, datetime
    
def get_current_time(location)->str:
    """
    Get the current time by location.

    Args:
        location (str) need to be in format of 'Europe/London'

    Returns:
        str: The current time in the specified location.

    Example:
        get_current_time('Europe/London')
    """
    try:
        location_time = datetime.now(pytz.timezone(location))
        return location_time.strftime("%Y-%m-%d %H:%M:%S")
    except Exception as e:
        return f"Error: {str(e)}"

tools = [query_wikipedia,get_stock_data,search_engine,get_current_time]

In [6]:
def run_agentic_from_tool(question,tools,debug=False):
    # print(get_tools_prompt(tools))
    thought = gen_thought(question)
    action = pick_tool(question,thought,tools)
    if debug:
        print(f"action:{action}")
    answer = use_tool(action)
    if debug:
        print(f"tool result:{answer}")
    summary = gen_summary_from_knowledge(question,answer)
    return summary

In [11]:
wx.connect()

answer = run_agentic_from_tool("compare stock price of IBM and MSFT",tools)
print(answer)

calling search_engine({'query': 'IBM stock price'})...
calling search_engine({'query': 'Microsoft stock price'})...
Error: Failed to retrieve data from the API. Status code: 429
calling get_current_time({'location': 'New York'})...
I'm sorry, I don't have the ability to compare stock prices directly. However, based on the information provided, the current stock price of IBM is 172.95 USD, while Microsoft (MSFT) is currently priced at 429 USD. Please note that these prices are subject to change and it's always recommended to check the most recent and reliable sources for accurate information.


In [11]:
wx.connect()

# answer = query_wikipedia("olympics")
# # print(answer)

# answer = gen_naive_answer( "who win the 47th president of USA")
# print(answer)

# answer = gen_answer_from_knowledge("who win the 47th president of USA",tools,debug=True)
# print(answer)

answer = run_agentic_from_tool("who win the 47th president of USA",tools)
print(answer)

calling search_brave({'query': '47th presidential election winner'})...
Based on the information provided, the 47th president of the USA is Donald Trump. He defeated Kamala Harris in the election.


In [12]:
answer = gen_naive_answer( "when it is now in hong kong")
print(answer)

answer = gen_answer_from_knowledge("how many hour different of hong kong to SFO",tools)
print(answer)

answer = run_agentic_from_tool("how many hour different of hong kong to SFO",tools)
print(answer)

NameError: name 'gen_naive_answer' is not defined

In [33]:
answer = gen_naive_answer("compare UK and france")
print(answer)

answer = gen_answer_from_knowledge("compare UK and france",tools)
print(answer)

answer = run_agentic_from_tool("compare UK and france",tools)
print(answer)

Sure, I can help with that. Here's a brief comparison between the UK and France:

1. **Geography**: The UK is an island nation located off the northwest coast of Europe. It consists of four countries: England, Scotland, Wales, and Northern Ireland. France, on the other hand, is the largest country in Western Europe and shares borders with several countries including Belgium, Luxembourg, Germany, Switzerland, Italy, Monaco, Spain, and Andorra.

2. **Population**: As of 2021, the estimated population of the UK is around 67 million, while France has a population of approximately 67 million as well.

3. **Language**: The official language of the UK is English, although there are also other languages spoken in Scotland, Wales, and Northern Ireland. French is the official language of France.

4. **Government**: The UK is a constitutional monarchy with a parliamentary system of government. France is a semi-presidential representative democratic republic.

5. **Economy**: Both countries have s

UK: Period 'daily' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']


calling get_stock_data({'ticker': 'FRA', 'period': 'daily', 'interval': '1d'})...


FRA: Period 'daily' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']


calling search_brave({'query': 'UK vs France'})...
calling clock({'location': 'London'})...
calling clock({'location': 'Paris'})...
Based on the provided knowledge, here are some comparisons between the United Kingdom and France:

1. **Geographical Size**: The United Kingdom is smaller than France. The UK covers approximately 244,376 square kilometers, while France covers around 643,801 square kilometers.

2. **Population**: Both countries have large populations. The UK has over 68.2 million people, while France has around 68.4 million people.

3. **History**: Both countries have rich histories and have played significant roles in shaping the world. The UK has a long history of parliamentary democracy and has been a leader in literature, science, and technology. France, on the other hand, has a history of influencing art, philosophy, and fashion.

4. **Economy**: Both countries have strong economies. The UK is a developed country with a high nominal per capita income. France also has a

In [ ]:
answer = gen_naive_answer("compare IBM and AMAZON stock price today")
print(answer)

answer = gen_answer_from_knowledge("compare IBM and AMAZON stock price today",tools)
print(answer)

answer = run_agentic_from_tool("compare IBM and AMAZON stock price today",tools)
print(answer)

To compare the stock prices of IBM and Amazon (AMZN) for today, I'll need to check the current market data. Please note that stock prices can fluctuate throughout the day. Here's a brief comparison based on the latest available data:

1. IBM (IBM):
   - Stock Price: $135.00 (as of the latest available data)
   - Market Capitalization: Approximately $128 billion
   - Industry: Technology (Hardware & Software)

2. Amazon (AMZN):
   - Stock Price: $2,900.00 (as of the latest available data)
   - Market Capitalization: Approximately $1.6 trillion
   - Industry: Retail & E-commerce

Please verify these figures with a reliable financial news source or stock market platform, as they may have changed since my last update.
An error occurred: string index out of range



$AMAZON: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")


I'm sorry for the inconvenience, but I don't have real-time data or the ability to fetch current stock prices. Therefore, I can't compare the stock prices of IBM and Amazon at this moment. Please check a reliable financial news source or a stock market app for the most recent information.


In [102]:
answer = run_agentic_from_tool("who is competitor of vllm",tools)
print(answer)

Based on the information provided, there is no direct mention of competitors for VLLM. However, it is mentioned that Friendli Engine is faster than VLLM, and there are other platforms like OpenL... that are compared to VLLM in terms of LLM deployment. Additionally, there are alternative tools like Ollama and TGI that are mentioned in relation to VLLM. Therefore, while there may not be a specific list of competitors, there are certainly other tools and platforms that are used for similar purposes and can be considered as alternatives or competitors in a broader sense.


In [100]:
answer = run_agentic_from_tool("will it be typhoon tmr in hong kong",tools)
print(answer)

Based on the provided knowledge, there is no Tropical Cyclone Warning Signal in force in Hong Kong. Therefore, it is not expected to be a typhoon tomorrow in Hong Kong.


In [34]:
answer = gen_answer_from_knowledge("stock price of IBM compare to MSFT",tools)
print(answer)

answer = run_agentic_from_tool("stock price of IBM compare to MSFT",tools)
print(answer)

calling get_stock_data({'ticker': 'IBM'})...
calling get_stock_data({'ticker': 'MSFT'})...
calling compare_stocks({'stock1': '{}', 'stock2': '{}'})...
I'm sorry for the confusion, but the provided knowledge does not contain stock prices for both IBM and MSFT in a comparable format. Therefore, I cannot provide a direct comparison between their stock prices. However, based on the available data, we can see that IBM's stock prices ranged from $210.27 to $215.01, while MSFT's stock prices ranged from $416.00 to $429.33 during the given period. Please note that these are not the actual stock prices, but rather the ranges within which they fluctuated.
calling query_wikipedia({'topic': 'IBM'})...
calling query_wikipedia({'topic': 'Microsoft (MSFT)'})...
Based on the information provided, I can give you details about IBM but not Microsoft (MSFT). IBM, or International Business Machines Corporation, is a multinational technology company that has been responsible for several technological innova

In [ ]:
answer = gen_answer_from_knowledge("compare Mickey Mouse and Indiana Jones")
print(answer)

answer = gen_answer_from_knowledge("compare Trump and Biden")
print(answer)

answer = gen_answer_from_knowledge("tell me about star war")
print(answer)

answer = gen_answer_from_knowledge("tell me about beijing and paris on olympics")
print(answer)

answer = gen_answer_from_knowledge("tell me about TVB")
print(answer)

answer = gen_answer_from_knowledge("stock price of IBM")
print(answer)

answer = gen_answer_from_knowledge("what is official language of singapore")
print(answer)

# 4) solve problem by coding

In [35]:
import io
import sys
import traceback

def gen_code(requirement,code,plan,error,tools,debug=False):
    prompt = f"""<|user|>
please generate python code that execute the plan to fulfill the requirement.
-understand the requriement in detail.
-understand the tool provided.
-ensure the python generated is executable.
-clean up if any unneccessary characters.
-review the problem in the code and the error if provided.
-prevent the error from the code if provided.
-involve the tool provided directly without define it.
-understand the plan in detail. dont miss the steps.
-**DONT** use SaaS service which need API KEY.
-**DONT** catch error.
-raise error when hit problem.
-**DONT** request input.
-**DONT** throw SystemExit
-**DONT** generate main().
-**DONT** generate explanation.
-ensure print the answer to output explicitely.
-generate the code directly.
-generate the code only.
<<SYS>>
requirement: `{requirement}`
code with problem: `{code}`
plan: `{plan}`
error: `{error}`
-involve the tool provided directly without define it.
tools: `{get_tools_prompt(tools)}`
<</SYS>>
-dont generate explanation.
-dont generate note.
-generate the code directly only.
<|assistant|>
```python"""
    if debug:
        print(prompt)
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT).replace("```","")
    return answer

def create_local_namespace(tools):
    return {func.__name__: func for func in tools}

def run_code(module_import, function_call, tools, max_traceback_lines=5):
    """
    try to run the generated code to get the answer.
    """
    # Create StringIO objects to capture standard output and error output
    output_stream = io.StringIO()
    error_stream = io.StringIO()

    # Redirect stdout and stderr
    old_stdout = sys.stdout
    old_stderr = sys.stderr
    sys.stdout = output_stream
    sys.stderr = error_stream

    output = None
    error = None

    # Using exec to import the module and evaluate the function call
    try:
        local_namespace = create_local_namespace(tools)
        exec(module_import, local_namespace, local_namespace)  # Dynamically import the module
        # print(f"|{function_call}|")
        exec(function_call, local_namespace, local_namespace)  # Evaluate the function call
    except ImportError as e:
        error = f"Import Error: {str(e)}"
    except Exception as e:
        # Capture the traceback for more detailed error information
        full_traceback = traceback.format_exc().strip()
        # Limit the traceback to the last 'max_traceback_lines' lines
        limited_traceback = "\n".join(full_traceback.splitlines()[-max_traceback_lines:])
        error = f"Unexpected Error: {str(e)}\n{limited_traceback}"
    finally:
        # Reset stdout to its original state
        sys.stdout = old_stdout
        sys.stderr = old_stderr

    # Get the output from the output_stream
    output = output_stream.getvalue().strip()
    error_output = error_stream.getvalue().strip()

    # Check the output or error
    if error:
        return f"Output: {output}", f"Error: {error}"
    elif error_output:
        return f"Output: {output}", f"Error Output: {error_output}"
    else:
        return f"Output: {output}", None
    
def gen_eval(question,stdout):
    prompt = f"""<|user|>classify if stdout include error.
-show sucess or fail only, but nothing else.
-show success if include no error. and end the generation.
-show fail if include error. and end the generation.
<<SYS>>
question: `{question}`
stdout: `{stdout}`
<</SYS>>
<|assistant|>success or fail:"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT).replace("```","")
    return answer
    
def gen_code_summary(requirement,code,output):
    prompt= f"""<|user|>
please generate a complete sentence summary base on the answer provided only.
-dont use your common sense.
-declare the assumption you made.
-only base on the requirement and answer provided.
-the output comes from the code execution.
-dont guess.
<<SYS>>
requirement: `{requirement}`
code: `{code}`
answer: `{output}`
-dont mention about code.
-use simple english.
<</SYS>>
<|assistant|>answer summary:"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT).replace("```","").replace("<|user|>","")
    return answer

def run_agentic_with_coding(requirement, max_turns=10, debug=False):
    """
    gen thought or plan first.
    for each turns:
        gen code base on plan until output is error free
    gen summary
    """
    thought = gen_thought(requirement)
    if debug:
        print(f"thought: {thought}")

    plan = thought

    code = ""
    error = ""
    # Loop until the review output is empty
    for turn in range(max_turns):
        print(f"turn #{turn+1} ---")
        # output = exec_with_output(code)
        code = gen_code(requirement, code , plan, error, tools)
        if debug:
            print(f"code: {code}")
        output, error = run_code("",code, tools)
        if debug:
            print(f"output: {output}")
        
        if error is None:
            break
        else:
            if debug:
                print(f"error {error}")
        # review = eval_output(requirement, output)
        # if debug:
        #     print(f"#4: {review}")
        
        # if "success" in review:  # Exit if review output is empty
        #     # print(f"#final result: {output}")
        #     break

    summary = gen_code_summary(requirement, code, output)
    return summary

In [36]:
output = run_agentic_with_coding("tell me is it only 100 prime number with in 10000?")
print(output)

output = run_agentic_with_coding("tell me is it only 100 prime number with in 1000?")
print(output)

output = run_agentic_with_coding("tell me all prime number with in 1000?")
print(output)

output = run_agentic_with_coding("tell me all prime number with in 1000? and it need to be ends with 7")
print(output)

output = run_agentic_with_coding("how many prime number with in 10000")
print(output)

output = run_agentic_with_coding("how many R in straberry")
print(output)

output = run_agentic_with_coding("how many r in straberry")
print(output)

output = run_agentic_with_coding("how many r/R in straberry")
print(output)

output = run_agentic_with_coding("how many 'a' in canada")
print(output)

output = run_agentic_with_coding("how many A in 'canada', dont check case")
print(output)

output = run_agentic_with_coding("how many a or A in the word canada")
print(output)

output = run_agentic_with_coding("in math, compare 9.11 with 9.8")
print(output)

output = run_agentic_with_coding("9.1 9.11 and 9.21, 9.8, which is bigger? and which is smaller")
print(output)

output = run_agentic_with_coding("compare 9.11 with 9.8")
print(output)

output = run_agentic_with_coding("which of 9.11 9.8 is bigger")
print(output)

output = run_agentic_with_coding("compare 9.11 with 9.A")
print(output)

output = run_agentic_with_coding("tell me Korea time")
print(output)

output = run_agentic_with_coding("tell me how is the different of Korea time and Hong Kong time")
print(output)

output = run_agentic_with_coding("tell me Hong Kong time")
print(output)

output = run_agentic_with_coding("search Hong Kong on wiki",debug=True)
print(output)

output = run_agentic_with_coding("tell me about Paris",debug=True)
print(output)

turn #1 ---
 there are more than 100 prime numbers within 10000.
turn #1 ---
 there are 168 prime numbers within the range of 1 to 1000.
turn #1 ---
 The prime numbers between 1 and 1000 are 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997